# 미래 수출 예측
- diff : 1년뒤 수출 - 현재년도 수출 
- label : diff가 0보다 크면 1(상승), 0보다 작거나 같으면 0(하락,유지)

In [31]:
import os
import numpy as np
import pandas as pd
from IPython.display import display
from tqdm import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', family='AppleGothic')
plt.rc('axes', unicode_minus=False)

import warnings
warnings.filterwarnings('ignore')

import collections
import seaborn as sns

In [32]:
import pickle
with open('../processed/country_to_idx.pkl', 'rb') as f:
    country_to_idx = pickle.load(f)

with open('../processed/idx_to_country.pkl', 'rb') as f:
    idx_to_country = pickle.load(f)

In [33]:
export = pd.read_pickle('../processed/export_GDP_pop_fx_wti.pkl')
export

,year,country,품목코드,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI
0,2012,Afghanistan,3.0,1.0,20203572.96,4.949620e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
1,2012,Afghanistan,4.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
2,2012,Afghanistan,7.0,2.0,20203572.96,9.899239e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
3,2012,Afghanistan,8.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
4,2012,Afghanistan,10.0,12.0,20203572.96,5.939543e-07,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128388,2022,Zimbabwe,90.0,299.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000
128389,2022,Zimbabwe,94.0,21.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000
128390,2022,Zimbabwe,95.0,2.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000
128391,2022,Zimbabwe,96.0,0.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000


In [34]:
export.year.unique()

array([2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [35]:
export['next_year'] = export.year + 1
export_temp2 = export[['country', 'year', '품목코드', '수출금액']]
export_diff = pd.merge(export, export_temp2, left_on=['country', 'next_year', '품목코드'], 
                       right_on=['country', 'year', '품목코드'], how='left')
export_diff['diff'] = export_diff['수출금액_y'] - export_diff['수출금액_x']
export_diff['label'] = export_diff['diff'].apply(lambda x: 1 if x>0 else 0)
export_diff.drop(['next_year','year_y','수출금액_y','diff'], axis=1, inplace=True)
export_diff.rename(columns={'year_x':'year', '수출금액_x':'수출금액'}, inplace=True)
export_diff

,year,country,품목코드,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI,label
0,2012,Afghanistan,3.0,1.0,20203572.96,4.949620e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
1,2012,Afghanistan,4.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
2,2012,Afghanistan,7.0,2.0,20203572.96,9.899239e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
3,2012,Afghanistan,8.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
4,2012,Afghanistan,10.0,12.0,20203572.96,5.939543e-07,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128388,2022,Zimbabwe,90.0,299.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000,0
128389,2022,Zimbabwe,94.0,21.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000,0
128390,2022,Zimbabwe,95.0,2.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000,0
128391,2022,Zimbabwe,96.0,0.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000,0


In [37]:
export_diff.to_csv('../processed/export_diff_label.csv', index=False)